# Modeling the focal plane of a large-format groundbased astronomical imaging instrument

This notebook shows how to fit data from a large-format camera with a parsimonious model that accounts for the optical distortion terms and chip gaps.

**Author**: David Shupe, Caltech/IPAC

Refer to: SciPy 2018 talk

Here are the steps:

1. Retrieve a set of PSF-fit catalogs from public ZTF data, for a single exposure.
2. Combine these catalogs into a single table with five columns for RA & Dec, local x-pixel and y-pixel, and chip number.
3. Use the statsmodels package to fit a model with terms for chip gaps, small rotations between chips, and overall optical distortion.
4. Write out a `.ahead` file for use with SCAMP from the Astromatic suite.

## Imports (all here to make sure we have them)

In [7]:
import numpy as np
from astropy.io import fits
from astropy.table import Table
from astropy.wcs import WCS
from astropy.utils.data import download_file
import pandas as pd
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns

## Download catalogs for a single science exposure for an extragalactic field

We need a ZTF exposure that is public and that contains data for all 64 quadrants.

Data meeting these requirements are available at https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2019/0408/164213/

In [18]:
template_url = ('https://irsa.ipac.caltech.edu/ibe/data/ztf/products/sci/2019/0408' +
                 '/164213/ztf_20190408164213_000747_zr_c01_o_q1_psfcat.fits')

Download the image corresponding to the template catalog and extract some metadata

In [19]:
sci_header = fits.getheader(template_url.replace('_psfcat.fits', '_sciimg.fits'))

In [14]:
sci_header.get('TELRAD')

110.128

In [15]:
sci_header.get('TELDECD')

47.7503

Make a WCS with a projection center at  crval1=TELRAD, crval2=TELDECD, with cdelt1 = cdelt2 = 1.0 degrees and a TAN projection.

In [17]:
proj_wcs = WCS(naxis=2)
proj_wcs.wcs.ctype = ['RA---TAN', 'DEC--TAN']
proj_wcs.wcs.crval = [sci_header.get('TELRAD'), sci_header.get('TELDECD')]
proj_wcs.wcs.cdelt = [1.0, 1.0] # one degree per "pixel"
proj_wcs.wcs.crpix = [4.5, 4.5]
proj_wcs.array_shape = [8, 8] # NAXIS2, NAXIS1

Use the template catalog to download and get columns from all the PSF catalogs.

In [20]:
psfcat = Table.read(download_file(template_url), format='fits')

In [21]:
psfcat.columns

<TableColumns names=('sourceid','xpos','ypos','ra','dec','flux','sigflux','mag','sigmag','snr','chi','sharp','flags')>

In [25]:
xvals = []
yvals = []
etavals = []
nuvals = []
rcids = []
ras = []
decs = []

for ccd in range(1, 17):
    for quadrant in range(1, 5):
        psf_url = template_url.replace('_c01_o_q1_psfcat.fits', f'_c{ccd:02}_o_q{quadrant:01}_psfcat.fits')
        fname = download_file(psf_url, cache=True)
        header = fits.getheader(fname)
        tab = Table.read(fname, format='fits')
        plane_coords = proj_wcs.wcs_world2pix(np.vstack([tab['ra'],tab['dec']]).T, 1)
        rcids.append(header['rcid']*np.ones_like(tab['ra']))
        xvals.append(tab['xpos'])
        yvals.append(tab['ypos'])
        etavals.append(plane_coords[:,0])
        nuvals.append(plane_coords[:,1])
        ras.append(tab['ra'])
        decs.append(tab['dec'])


## Fit a linear model

## Form global pixel coordinates and fit a quadratic model

## Output a .ahead file for SCAMP